# Packets of thought (NLP概览）

## 1.4 计算机所能理解的“语言”

最简单的方式是创建一系列的规则，使用各种`if ... else ...`表示。这等价于编写一种特殊的程序，即**优先状态机**（FSM，finite state machine）。如果你使用过正则表达式，那么你已经使用过FSM了。FSM是NLP中的方法之一。

纯粹根据语句的形式（基于一组规则）来判断，我们要用到**形式语言(formal language)**。形式语言是自然语言的子集。

形式语言与自然语言的区别：
编程语言的主要特点：
* 上下文无关语言
* 使用上下文无关文法解析，解析颇为高效
* 正则语言也可以高效解析
自然语言：
* 不是正则的
* 不是上下文无关的
* 不能使用形式语法来定义

### 1.4.3 一个简单的chatbot

该bot基于模式匹配，此方法在现代的ML方法之前是很常见的，包括Amazon Alexa之类的chatbot亦使用此方法。

可以看到，尽管这里脑补了较为复杂的模式，还是很难避免出现两类问题：false negative和false positive。一方面，可说它太严格了，另一方面，也可说它太宽松了。

由于算力的缺乏，早期NLP研究者必须使用人脑来弥补，添加很多复杂的人工规则。这类方法被称为pattern-base approach，它们不仅可以匹配字符序列，还可以匹配更高级别的模式，如词序列、词性等。stemmer、tokenizer及ELIZA这样的对话引擎都是基于这类方法的。

In [5]:
r = r"[^a-z]*([y]o|[h']?ello|ok|hey|(good[ ])?(morn[gin']{0,3}|"\
    r"afternoon|even[gin']{0,3}))[\s,;:]{1,3}([a-z]{1,20})"
re_greeting = re.compile(r, flags=re.I)

In [7]:
re_greeting.match('Hello Rosa')

<_sre.SRE_Match object; span=(0, 10), match='Hello Rosa'>

In [8]:
re_greeting.match('Morning Rosa')

<_sre.SRE_Match object; span=(0, 12), match='Morning Rosa'>

In [15]:
# 输出something

my_names = {'rosa', 'rose', 'chatbot', 'bot'}
curt_names = {'hal', 'you', 'u'}
greeter_name = 'NLP'


def generate(s):
    m = re_greeting.match(s)
    if m:
        at_name = m.groups()[-1]
        if at_name in curt_names:
            print('good one')
        elif at_name.lower() in my_names:
            print(f'hi {greeter_name}, how are you?')

In [16]:
generate('hello Rosa')

hi NLP, how are you?


### 1.4.4 另一种方法

除了基于规则的方法，如果我们有足够多的数据，是否可以基于统计来完成对话系统呢？最直接的想法是，看一下当前问题在数据库中有没有，如果有，那么从已有的回答中，返回一条。这种方法的问题是，一个问句，要么“在”要么“不在”数据库中，它很容易受拼写错误和同义词之类的影响。我们需要一个合理的方法，来度量两个句子**语义**的“相似度”。

单纯从字面的字符来判断语义相似度很容易出问题，即使借Jaccard、Levenshtein、Euclidean向量距离之类的方法，也难以有大的改观。不过如基于规则的方法一样，此类方法也有它适合的领域，如拼写检查、专有名词提取等。如果我们关注语义胜于拼写，那么需要考虑其它更好的方法。

## 1.5 hyperspace 一瞥

第三章中将涉及**cosine distance**，第四章将借助它来完成向量的降维。

## 1.6 词序与语法

词序很重要，语言中与词序有关的规则称为“语法”。如果使用词袋模型，那么词序信息就丢失了。在实现自然语言查询之类的应用时，词袋不是一个很好的选择，幸运的是，目前的主流工具在语法树解析上有很高的准确率，如Spacy有93%，SyntaxNet有94%。

## 1.7 chatbot NLP pipeline

chatbot的pipeline需要四个处理阶段（stage），也需要有数据库维护过去的对话与响应。每个阶段都可能包含一个或多个处理算法，以并行或串行的方式执行。

* Parse: 从自然语言文本中提取特征、结构化数据
* Analyze：通过情感、语法和语义来泛化、合并特征
* Generate：通过模板、搜索和语言模型生成响应信息
* Execute：基于对话历史与目标计划语句，并选择下一个响应

深度学习和数据驱动编程使得NLP和chatbot的应用愈发广泛。

# 2、构造词汇表

* tokenizing text
* 处理不标准的表达与表情符号，如来自社交媒体的那些
* stemming、lemmatization
* 构造语句的向量表示
* 构造sentiment analyzer

本章主要讨论如何将文档（任意字符串）分解为离散的、具有一定意义的token。

基本的token近似于*词（word）*，但一般来说，token不一定是词。比如”ice cream“，我们更愿意将其作为一个整体来看待。另一方面，词甚至还能进一步分解为音节（syllable）、前缀、后缀，乃至字母（letter/grapheme）。

词可能是”隐含的“或”不可见的“，不如当你说出”Don't“时。

与词相关的概念是*n-grams*。为完整起见，这里的方法将保留所有的词以及n-grams，但一般来说，特征提取很少会保留所有信息，至于保留多少这就是NLP中的艺术了。

## 2.1 stemming的挑战

stemming是指把同一个词的不同屈折形式归为一类，某些人花费了毕生精力来完成此事。考虑一个例子，后缀”ing“对于以下几个词：ending、running、sing，或复数形式的”s“对于以下几个词：words、bus、lens。这一问题的解决办法，一是传统的基于规则，二是基于统计方法，有了足够多的数据，可以完全避免人工规则。

## 2.2 通过tokenizer构建词汇表

在NLP中，标记化（tokenization）是一种特殊的文档分割（segmentation）过程。segmentation将文本分割为更小的部分，如段落、句子、短语或token（通常为词）。顾名思义，tokenization就是将文本分割为token。

作为NLP的基础组块，与编译器中的某些部分可视为等价物：

* tokenizer：scanner、lexer
* vocabulary：lexicon
* parser：compiler
* token、term、word、n-gram：token、symbol

tokenization是NLP pipeline的第一步，对后续步骤可能会产生巨大影响。tokenizer将非结构化的数据转化为离散的信息块，这些块的**统计信息**本身即形成一个向量，从而可用于某些ML算法。BOW向量适合于搜索之类的应用。

最简单的方法是使用空白字符分割：


In [17]:
sent = "Thomas Jefferson began building Monticello at the age of 26."
sent.split()

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

在上例中，除了末尾的句号，其它部分是没问题的。先不管它，现在考虑至此的第一种向量表示法：**one-hot vectors**，它可以将文本表示为”数值“：

In [21]:
import numpy as np

tokens = sent.split()
vocab = sorted(set(tokens))
print('vocab:', ' '.join(vocab))

n_tokens = len(tokens)
n_vocab = len(vocab)

one_hot_vecs = np.zeros((n_tokens, n_vocab), int)
for i, w in enumerate(tokens):
    one_hot_vecs[i, vocab.index(w)] = 1
    
one_hot_vecs

vocab: 26. Jefferson Monticello Thomas age at began building of the


array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

这里的关键信息是，矩阵的维度是(n_tokens, n_vocab)，即行的顺序与tokens顺序一致，每一行表示相应的token信息，在一行上值为1的那个元素表示token是哪一个。通过这个矩阵及`vocab`表，确实可以保存下句子的完整信息。这个矩阵亦可以通过`DataFrame`表示：

In [22]:
import pandas as pd

pd.DataFrame(one_hot_vecs, columns=vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


hot的意思是”on“，每一行只有一个元素为1，所以是one-hot。该方法得到的数据是无损的，通常可用于NN、seq-to-seq语言模型、生成式语言模型。但是这个表无疑是太大了，尤其是在列上，词汇表可能会扩大到数以百万计。这样，即使只是保存几千本书，可能就需要数百T的容量。

如果放弃”完整信息“，比如讲所有行相加，那么就得到了BOW向量，或词频向量，它仅关心词的发生频率，忽略了顺序。

如果需要实现关键词搜索，只要对两个向量做OR运算。下面是bow的简单实现：

In [24]:
sent_bow = {token: 1 for token in tokens}
sorted(sent_bow.items())

[('26.', 1),
 ('Jefferson', 1),
 ('Monticello', 1),
 ('Thomas', 1),
 ('age', 1),
 ('at', 1),
 ('began', 1),
 ('building', 1),
 ('of', 1),
 ('the', 1)]

词袋模型，除了只用一个向量即可表示文档，还有一个极大的好处时，它只保留出现的词，不用考虑整个词汇表。这个dict还可以进一步压缩，比如将每个词表示一个整型值（它的索引），Spacy就是这么做的。如上面用`DataFrame`类似，词袋可以表示为`Series`：

In [27]:
pd.DataFrame(pd.Series(sent_bow), columns=['sent']).T

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent,1,1,1,1,1,1,1,1,1,1


In [29]:
# 现在可以使用多一点文档了
sents = ['Thomas Jefferson began building Monticello at the age of 26.',
         'Construction was done mostly by local masons and carpenters.',
         'He moved into the South Pavilion in 1770.',
         "Turning Monticello into a neoclassical masterpiece was Jefferson's obsession."]

corpus = {}

for i, sent in enumerate(sents):
    corpus[f'sent{i}'] = {token: 1 for token in sent.split()}
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df[df.columns[:10]]

,1770.,26.,Construction,He,Jefferson,Jefferson's,Monticello,Pavilion,South,Thomas
sent0,0,1,0,0,1,0,1,0,0,1
sent1,0,0,1,0,0,0,0,0,0,0
sent2,1,0,0,1,0,0,0,1,1,0
sent3,0,0,0,0,0,1,1,0,0,0


现在有了多个文档的向量表示，那么有时需要比较它们的相似度，最基本的方法会用到**点积（dot product）**。

### 2.2.2 BOW的相似度

In [31]:
df2 = df.T
df2.sent0.dot(df2.sent1), df2.sent0.dot(df2.sent3)

(0, 1)

这里点积的值表明两个文档间有多少个共同的token。BOW是接触到的第一种向量空间模型（VSM），除了点积，它还可以进行加法、减法、OR、AND等操作，后者对于搜索之类的应用有主要的意义。

### 2.2.3 更好的token

In [34]:
import re

spliter = re.compile(r'[-\s.,;!?]+', re.U)

sent = 'Thomas Jefferson began building Monticello at the age of 26.'
tokens = spliter.split(sent)
tokens = [t for t in tokens if t and t not in '- \t\n.,;!?']
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

对于英语，几个常见的库都包含了tokenizer，如`spaCy`、`Stanford CoreNLP`、`NLTK`，后两个主要用于学术领域，spaCy则是工业级的。

值得一提的是，NLTK还包含了一个`casual_tokenize`，可用于Twitter、Facebook等社交网站的文本。

In [1]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+|\S+')

sent = 'Thomas Jefferson began building Monticello at the age of 26.'
tokenizer.tokenize(sent)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '.']

In [3]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

sent = "Monticello wasn't designated as UNESCO World Heritage Site until 1987."

# treebank tokenizer可以处理contraction的情形
tokenizer.tokenize(sent)

['Monticello',
 'was',
 "n't",
 'designated',
 'as',
 'UNESCO',
 'World',
 'Heritage',
 'Site',
 'until',
 '1987',
 '.']

### 2.2.4 通过n-grams扩展词汇

仅通过token（unigram），无法判断一个同时包含`ice`和`cream`的文档是否真地与`ice cream`有关，换言之，它丢失了所有与词序有关的信息，而n-grams保留了少许。通常，我们可以说，n-grams保留了一定的**上下文**信息。

需要考虑的是，n-grams不一定是”有效的“复合词或词组，它仅仅表明某些词共同出现（共现）了。

In [4]:
spliter = re.compile(r'[-\s.,;!?]+', re.U)

sent = 'Thomas Jefferson began building Monticello at the age of 26.'
tokens = spliter.split(sent)
tokens = [t for t in tokens if t and t not in '- \t\n.,;!?']
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

In [6]:
from nltk.util import ngrams

list(ngrams(tokens, 2))

[('Thomas', 'Jefferson'),
 ('Jefferson', 'began'),
 ('began', 'building'),
 ('building', 'Monticello'),
 ('Monticello', 'at'),
 ('at', 'the'),
 ('the', 'age'),
 ('age', 'of'),
 ('of', '26')]

一般来说，如果token或n-grams出现频率极低，那么可认为它未携带足够的相关信息，这里的相关是指与文档的主题或分类有关。同时，token的组合数要比token数多很多，而如果特征向量的维度超过了文档数，那么特征提取这一步骤会给后续步骤带来不利影响，出现过拟合的情况。所以频率过低的n-grams一般会过滤掉。

另一方面，如果频率过高，我们也可以认为它是无区分度的，因此也会被忽略掉。这种词或n-grams通常称为**Stop Words**。

不过，在使用n-grams时，停用词不能够简单地去掉，否则会影响真正的n-grams，而且此时，去掉少数n-grams给内存等带来的收益不大。此时还是保留为好。

In [7]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andersc/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
stopwords = nltk.corpus.stopwords.words('english')
len(stopwords)

179

In [9]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stopwords
len(sklearn_stopwords)

318

### 2.2.5 normalize词汇表

减小词汇表大小可以减少过拟合的可能性。

* case folding: 是否统一大小写，取决于当前应用（如NER等）
* stemming：对搜索来说，增加了recall，降低了precision。英语中最流行的两个stemming算法是`Porter`和`Snowball`。
* lemmatization：lemma意为词条，与词义有关。由于它考虑了词义，而不仅仅是后缀，因此大部分时候好于stemming，而且lemma都是合法的词。因此建议先lemmatization，再stemming。

三种方法都可以缩小词汇量，增加recall，降低precision。底线是，如果不是数据量很有限，避免使用：）

In [13]:
def stem(phrase):
    return ' '.join([re.findall('^(.*ss|.*?)(s)?$', word)[0][0].strip("'") for word in phrase.lower().split()])

stem('houses'), stem("Doctor House's calls")

('house', 'doctor house call')

In [16]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

sent = "dish washer's washed dishes"
' '.join([stemmer.stem(w).strip("'") for w in sent.split()])

'dish washer wash dish'

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/andersc/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## 2.3 情感（Sentiment）

以上将文本分割为token或n-grams，它们多少会包含一点“信息”，信息的一个主要方面是其表达的情感——总体的感觉或情绪。**情感分析（sentiment analysis）**是NLP的常见应用之一。

比如一家公司希望了解用户如何看到其产品。一种方式是让用户打星（豆瓣），另一更自然的方式是让用户输入一段文字作为评论，此时情感分析就很有用了。

情感分析有两种方法：

* 基于人工规则：使用启发式（heuristics）方法
* 机器学习方法：常使用来自tweet等网站的数据，因为那些数据会带有hashtag。对于豆瓣或淘宝，可以结合用户的评论文字与其打星。

### 2.3.1 VADER - 基于规则

VADER是较早出现的基于规则的情感分析工具。

[vaderSentiment](https://github.com/cjhutto/vaderSentiment)

### 2.3.2 Naive Bayes

NB相当于通过数据学习出类似于VADER中的系数。

In [23]:
movies = pd.read_csv('movieReviewSnippets_GroundTruth.txt', delimiter='\t', index_col='id', error_bad_lines=False)
movies.head().round(2)

,sentiment,text
id,,
1,2.27,The Rock is destined to be the 21st Century's ...
2,3.53,The gorgeously elaborate continuation of ''The...
3,-0.60,Effective but too tepid biopic
4,1.47,If you sometimes like to go to the movies to h...
5,1.73,"Emerges as something rare, an issue movie that..."


In [24]:
movies.describe().round(2)

,sentiment
count,10605.00
mean,0.00
std,1.92
min,-3.88
25%,-1.77
50%,-0.08
75%,1.83
max,3.94


In [26]:
from collections import Counter

from nltk.tokenize import casual_tokenize

bow = []
for text in movies.text:
    bow.append(Counter(casual_tokenize(text)))
    
df_bows = pd.DataFrame.from_records(bow)
df_bows = df_bows.fillna(0).astype(int)
df_bows.shape

(10605, 20756)

In [56]:
df_bows.head()

,!,"""",#,$,%,&,',(,),*,...,zips,zombie,zombies,zone,zoning,zzzzzzzzz,½,élan,–,’
0,0,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb = nb.fit(df_bows, movies.sentiment > 0)

In [50]:
movies['pred'] = nb.predict_proba(df_bows)[:, 1] * 8 - 4
movies['error'] = (movies.pred - movies.sentiment).abs()

In [51]:
movies.error.mean().round(1)

1.9

In [52]:
movies['pos'] = (movies.sentiment > 0).astype(int)
movies['pred_pos'] = (movies.pred > 0).astype(int)

In [53]:
movies.head(8)

,sentiment,text,pred,error,pos,pred_pos
id,,,,,,
1,2.266667,The Rock is destined to be the 21st Century's ...,2.511515,0.244848,1,1
2,3.533333,The gorgeously elaborate continuation of ''The...,3.999904,0.466571,1,1
3,-0.600000,Effective but too tepid biopic,-3.655976,3.055976,0,0
4,1.466667,If you sometimes like to go to the movies to h...,1.940954,0.474287,1,1
5,1.733333,"Emerges as something rare, an issue movie that...",3.910373,2.177040,1,1
6,2.533333,The film provides some great insight into the ...,3.995188,1.461854,1,1
7,2.466667,Offers that rare combination of entertainment ...,3.960466,1.493799,1,1
8,1.266667,Perhaps no picture ever made has more literall...,-1.918701,3.185368,1,0


In [55]:
precision = (movies.pos == movies.pred_pos).sum() / len(movies)
precision

0.9344648750589345

需要注意的点：

* NB对于否定词的处理不够好，需要引入n-grams；
* 需要区分开训练集和测试集
* 在movies上训练后，对于其它领域的效果会差很多

# 3、词的数学（TF-IDF向量）

* 统计词频以分析语义
* 通过Zipf法则预测词的概率
* 词的向量表示及用法
* 使用IDF寻找“相关的”文档
* 使用余弦相似度估计文档的相似度

在将文本分割为token后，可以想象的是，对于一个文档而言，不同token的重要性不同。本章将讨论衡量token重要性的一种方法，该方法在搜索引擎和垃圾邮件过滤上是传统的主流方法。

BOW向量基本上是**bit vector**，主要表示一个token是否出现。如果有办法将token表示为连续值，那么会有更有趣的数学操作可做。接下来将讨论三种方法：

* BOW：词频
* Bags of n-grams：
* TF-IDF

这三种方法都是基于频率的，都属于**浅层（shallow）**方法，但在某些应用中，它们已经足够强大，如邮件过滤和情感分析。

## 3.1 词袋模型

在上面的NB例子中，已经用到了词袋模型，它不是最简单的`one-hot`，而是包含了词频，其隐含的意思是，频率越高者对文档意义贡献越大。

In [57]:
from nltk.tokenize import TreebankWordTokenizer

tokenizer = TreebankWordTokenizer()

sent = 'The faster Harry got to the store, the faster Harry, the faster, would get home.'
tokens = tokenizer.tokenize(sent.lower())

bow = Counter(tokens)
bow

Counter({'the': 4,
         'faster': 3,
         'harry': 2,
         'got': 1,
         'to': 1,
         'store': 1,
         ',': 3,
         'would': 1,
         'get': 1,
         'home': 1,
         '.': 1})

词在文档中的频率称为*term frequency*，简称*TF*。由于不同文档间的长度相差会很大，因此通常要使用$count(term) / len(doc)$表示一个term的TF值。下面来看一个更大的文档：

In [59]:
kite_text = """
A kite is traditionally a tethered heavier-than-air craft with wing surfaces that react
against the air to create lift and drag. A kite consists of wings, tethers, and anchors.
Kites often have a bridle to guide the face of the kite at the correct angle so the wind
can lift it. A kite’s wing also may be so designed so a bridle is not needed; when
kiting a sailplane for launch, the tether meets the wing at a single point. A kite may
have fixed or moving anchors. Untraditionally in technical kiting, a kite consists of
tether-set-coupled wing sets; even in technical kiting, though, a wing in the system is
still often called the kite.

The lift that sustains the kite in flight is generated when air flows around the kite’s
surface, producing low pressure above and high pressure below the wings. The
interaction with the wind also generates horizontal drag along the direction of the
wind. The resultant force vector from the lift and drag force components is opposed
by the tension of one or more of the lines or tethers to which the kite is attached. The
anchor point of the kite line may be static or moving (such as the towing of a kite by
a running person, boat, free-falling anchors as in paragliders and fugitive parakites
or vehicle).

The same principles of fluid flow apply in liquids and kites are also used under water.
A hybrid tethered craft comprising both a lighter-than-air balloon as well as a kite
lifting surface is called a kytoon.

Kites have a long and varied history and many different types are flown
individually and at festivals worldwide. Kites may be flown for recreation, art or
other practical uses. Sport kites can be flown in aerial ballet, sometimes as part of a
competition. Power kites are multi-line steerable kites designed to generate large forces
which can be used to power activities such as kite surfing, kite landboarding, kite
fishing, kite buggying and a new trend snow kiting. Even Man-lifting kites have
been made.
"""

In [60]:
tokens = tokenizer.tokenize(kite_text.lower())
token_counts = Counter(tokens)
token_counts.most_common(5)

[('the', 26), ('a', 20), ('kite', 16), (',', 14), ('and', 10)]

In [61]:
stopwords = nltk.corpus.stopwords.words('english')

tokens = [t for t in tokens if t not in stopwords]
token_counts = Counter(tokens)
token_counts.most_common(5)

[('kite', 16), (',', 14), ('kites', 8), ('wing', 5), ('lift', 4)]

## 3.2 向量化

In [63]:
docs = ["The faster Harry got to the store, the faster and faster Harry would get home.",
        "Harry is hairy and faster than Jill.",
        "Jill is not as hairy as Harry."]

doc_tokens = []
for doc in docs:
    doc_tokens.append(sorted(tokenizer.tokenize(doc.lower())))
    
len(doc_tokens[0])

17

In [64]:
all_doc_tokens = sum(doc_tokens, [])
len(all_doc_tokens)

33

In [65]:
lexicon = sorted(set(all_doc_tokens))
len(lexicon)

18

In [66]:
from collections import OrderedDict

zero_vector = OrderedDict((token, 0) for token in lexicon)

In [68]:
import copy

doc_vectors = []
for doc in docs:
    vec = copy.copy(zero_vector)
    tokens = tokenizer.tokenize(doc.lower())
    token_counts = Counter(tokens)
    for k, v in token_counts.items():
        vec[k] = v / len(lexicon)
    doc_vectors.append(vec)
    
doc_vectors[0]

OrderedDict([(',', 0.05555555555555555),
             ('.', 0.05555555555555555),
             ('and', 0.05555555555555555),
             ('as', 0),
             ('faster', 0.16666666666666666),
             ('get', 0.05555555555555555),
             ('got', 0.05555555555555555),
             ('hairy', 0),
             ('harry', 0.1111111111111111),
             ('home', 0.05555555555555555),
             ('is', 0),
             ('jill', 0),
             ('not', 0),
             ('store', 0.05555555555555555),
             ('than', 0),
             ('the', 0.16666666666666666),
             ('to', 0.05555555555555555),
             ('would', 0.05555555555555555)])

以上操作，对于一个corpus，为其中的每个文档建立了向量表示。不过，`vec[k] = v / len(lexicon)`应该是有问题的？

### 3.2.1 向量空间

一个**向量空间**是其中所有向量的集合，如二维向量空间对应平面内的所有向量。（回忆一哈线性代数什么的）

NLP中，向量空间的维度是词汇表的大小，一般记为$K$或$|V|$。衡量向量的相似度，主要通过它们的方向，而非距离（欧几里得距离等），而方向就引出“余弦相似度”。

如果余弦相似度接近1，表明两个文档不仅包含类似的词，各个词的出现比例也是接近的。如果为0，则说明两个文档没有任何共同的词（但并不全然说明两个文档语义无关）。

In [69]:
nltk.download('brown')

[nltk_data] Downloading package brown to /Users/andersc/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [72]:
from nltk.corpus import brown

brown_words = brown.words()
print(len(brown_words))
brown_words[:10]

1161192


['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of']

## 3.4 Topic Modeling

单纯依靠$TF$不能了解一个词**相对于**其它文档对于当前文档的重要性。一个词在当前文档频率较高，但如果在多数文档中都是如此，那么它也就没那么重要了。这是就需要$IDF$了。

In [75]:
kite_into = kite_text
kite_history = """Kites were invented in China, where materials ideal for kite building were readily
available: silk fabric for sail material; fine, high-tensile-strength silk for flying line;
and resilient bamboo for a strong, lightweight framework.

The kite has been claimed as the invention of the 5th-century BC Chinese
philosophers Mozi (also Mo Di) and Lu Ban (also Gongshu Ban). By 549 AD
paper kites were certainly being flown, as it was recorded that in that year a paper
kite was used as a message for a rescue mission. Ancient and medieval Chinese
sources describe kites being used for measuring distances, testing the wind, lifting
men, signaling, and communication for military operations. The earliest known
Chinese kites were flat (not bowed) and often rectangular. Later, tailless kites
incorporated a stabilizing bowline. Kites were decorated with mythological motifs
and legendary figures; some were fitted with strings and whistles to make musical
sounds while flying. From China, kites were introduced to Cambodia, Thailand,
India, Japan, Korea and the western world.

After its introduction into India, the kite further evolved into the fighter kite, known
as the patang in India, where thousands are flown every year on festivals such as
Makar Sankranti.

Kites were known throughout Polynesia, as far as New Zealand, with the
assumption being that the knowledge diffused from China along with the people.
Anthropomorphic kites made from cloth and wood were used in religious ceremonies
to send prayers to the gods. Polynesian kite traditions are used by anthropologists get
an idea of early “primitive” Asian traditions that are believed to have at one time
existed in Asia."""

In [76]:
tokenizer = TreebankWordTokenizer()

intro_tokens = tokenizer.tokenize(kite_into.lower())
hist_tokens = tokenizer.tokenize(kite_history.lower())

intro_total, hist_total = len(intro_tokens), len(hist_tokens)
intro_total, hist_total

(365, 297)

In [77]:
intro_tf, hist_tf = {}, {}
intro_counts, hist_counts = Counter(intro_tokens), Counter(hist_tokens)

intro_tf['kite'] = intro_counts['kite'] / intro_total
hist_tf['kite'] = hist_counts['kite'] / hist_total

round(intro_tf['kite'], 2), round(hist_tf['kite'], 2)

(0.04, 0.02)

原始的IDF值会有一些问题。比如在一个很大的语料库中，一个出现1次的词IDF值是出现10次的词的10倍，但实际上两者都属于“很高”的那一类，而非相差如此悬殊。因此一般做法是取log。

看起来定义极为简单的$TF-IDF$是搜索引擎的基础。这里实现一个基本的计算方法：

$$ \frac{f_{t,d}}{\sum_{t' \in d}^{} f_{t',d} } * log(\frac{N}{1+n_t}) $$

这一算式计算文档d中term t的TF-IDF值，生产环境中可考虑sklean等库的实现。

In [78]:
docs = ["The faster Harry got to the store, the faster and faster Harry would get home.",
        "Harry is hairy and faster than Jill.",
        "Jill is not as hairy as Harry."]

doc_tokens = []
for doc in docs:
    doc_tokens.append(sorted(tokenizer.tokenize(doc.lower())))
    
all_doc_tokens = sum(doc_tokens, [])
lexicon = sorted(set(all_doc_tokens))
len(all_doc_tokens), len(lexicon)

(33, 18)

In [88]:
from collections import OrderedDict
from math import log

zero_vector = OrderedDict((token, 0) for token in lexicon)

import copy

N = len(doc_tokens) + 1
all_token_counts = Counter()
for tokens in doc_tokens:
    all_token_counts.update(set(tokens))

doc_vectors = []
for tokens in doc_tokens:
    n_doc_tokens = len(tokens)
    vec = copy.copy(zero_vector)
    token_counts = Counter(tokens)
    for k, v in token_counts.items():
        tf = v / n_doc_tokens
        idf = log(N / all_token_counts[k])
        vec[k] = tf * idf
    doc_vectors.append(vec)
    
doc_vectors[0]

OrderedDict([(',', 0.08154672712469944),
             ('.', 0.016922474850104754),
             ('and', 0.04077336356234972),
             ('as', 0),
             ('faster', 0.12232009068704917),
             ('get', 0.08154672712469944),
             ('got', 0.08154672712469944),
             ('hairy', 0),
             ('harry', 0.03384494970020951),
             ('home', 0.08154672712469944),
             ('is', 0),
             ('jill', 0),
             ('not', 0),
             ('store', 0.08154672712469944),
             ('than', 0),
             ('the', 0.24464018137409835),
             ('to', 0.08154672712469944),
             ('would', 0.08154672712469944)])

In [89]:
query = "How long does it take to get to the store?"

tokens = tokenizer.tokenize(query.lower())
n_query_tokens = len(tokens)

query_vec = copy.copy(zero_vector)
query_token_counts = Counter(tokens)
for k, v in query_token_counts.items():
    if k in all_token_counts:
        tf = v / n_query_tokens
        idf = log(N / (1 + all_token_counts[k]))
        query_vec[k] = tf * idf
    
query_vec

OrderedDict([(',', 0),
             ('.', 0),
             ('and', 0),
             ('as', 0),
             ('faster', 0),
             ('get', 0.06301338005090412),
             ('got', 0),
             ('hairy', 0),
             ('harry', 0),
             ('home', 0),
             ('is', 0),
             ('jill', 0),
             ('not', 0),
             ('store', 0.06301338005090412),
             ('than', 0),
             ('the', 0.06301338005090412),
             ('to', 0.12602676010180824),
             ('would', 0)])

In [90]:
import math


def cosine_sim(v1, v2):
    v1 = [v for v in v1.values()]
    v2 = [v for v in v2.values()]
    
    dot_prod = 0
    for i, v in enumerate(v1):
        dot_prod += v * v2[i]
    
    mag1 = math.sqrt(sum([x**2 for x in v1]))
    mag2 = math.sqrt(sum([x**2 for x in v2]))
    
    return dot_prod / (mag1 * mag2)

In [91]:
for i, doc_vec in enumerate(doc_vectors):
    print(i, cosine_sim(query_vec, doc_vec))

0 0.6115759372893524
1 0.0
2 0.0


In [87]:
len(query_vec), len(doc_vectors[0])

(18, 18)

# 4、寻找词频中的语义（语义分析）

* 分析语义，创建主题向量
* 使用主题向量进行语义搜索
* 使用语义组件作为pipeline的特征
* 探索高维向量空间

如果你知道想搜索的词具体是什么，那么TF-IDF是不错的选择，但很多时候并不清楚要搜什么。

人们开发了*LSA（Latent Semantic Analysis）*，LSA不仅可以表示词的向量还可以表示整个文档的向量。这一章即介绍**topic vector**。如此，可以进行语义搜索，而非词的搜索。另外，还可以识别出能够表示整个文档的词或n-grams（关键词提取）。

## 4.1 从词频到topic score

TF-IDF基于term的准确拼写，即使考虑到大小写、stemming、lemmatization之类，亦是如此。问题是，拼写类似的词语义未必类似，而同义之词拼写可能会差得很远。同时stemming不能处理好反义词的情况。

### 4.1.2 主题向量（topic vector）

对TF-IDF向量进行相加操作，只是得到另一个关于“词频”的向量，与语义不尽相关。我们需要一种更为“紧致的”向量表示——word-topic vector。对于这种向量，加减操作就蕴含着更多了。

注：**polysemy**：一词多义。

* Homonym：同形（音）异义词
* Zeugma：同一句中，一词具有多义

对于大型的corpus，TF-IDF向量可能高达百万维，现在需要将其压缩为数百维，同时向量还要能表示一个词对于某个主题的贡献为多少。

### 4.1.3 思想实验

假设有一个小型语料库，其中文档是关于三个已知主题的，那么可以为每个term分配权重，表示这个词对每个主题的贡献是多少。

### 4.1.4 一个算法

需要一个算法将TF-IDF向量转换为topic向量。

LSA算法分析TF-IDF矩阵将词收集到主题中，也可以用于bow，但前者效果更好。由于主题数量一般要远小于词汇大小，因此LSA常常被看作一种降维技术，奇妙的是它与另一种降维技术PCA遵循同样的数学原理。

LSA有两个近似算法：

* LDA：Linear Discriminant Analysis（线性判别分析）
* LDiA：Latent Dirichlet Allocation

### 4.1.5 An LDA Classifier

LDA是最直接、快速的降维和分类模型之一。它是监督式算法，因此需要标注数据，但所需数据远少于其它facier的算法。这里用它来解决一个二分类问题。

In [93]:
from nlpia.data.loaders import get_data

sms = get_data('sms-spam')

2019-09-22 00:55:42,872 WARNING:nlpia.constants:106:            <module> Starting logger in nlpia.constants...


In [94]:
index = [f'sms{i}{"!"*j}' for (i, j) in enumerate(sms.spam)]

In [95]:
sms = pd.DataFrame(sms.values, columns=sms.columns, index=index)
sms['spam'] = sms.spam.astype(int)
len(sms)

4837

In [96]:
sms.spam.sum()

638

In [97]:
sms.head()

,spam,text
sms0,0,"Go until jurong point, crazy.. Available only ..."
sms1,0,Ok lar... Joking wif u oni...
sms2!,1,Free entry in 2 a wkly comp to win FA Cup fina...
sms3,0,U dun say so early hor... U c already then say...
sms4,0,"Nah I don't think he goes to usf, he lives aro..."


In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

tfidf_vectorizer = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf = tfidf_vectorizer.fit_transform(raw_documents=sms.text).toarray()
tfidf.shape

(4837, 9232)

特征数远超文档数，更远超spam数量，因此模型没有足够信息确定文档是否为spam，这时NB分类器不敷使用。

In [101]:
mask = sms.spam.astype(bool).values
spam_centroid = tfidf[mask].mean(axis=0)
spam_centroid.round(2)

array([0.06, 0.  , 0.  , ..., 0.  , 0.  , 0.  ])

In [102]:
ham_centroid = tfidf[~mask].mean(axis=0)
ham_centroid.round(2)

array([0.02, 0.01, 0.  , ..., 0.  , 0.  , 0.  ])

In [103]:
# line between centroids
spamminess_score = tfidf.dot(spam_centroid - ham_centroid)
spamminess_score.round(2)

array([-0.01, -0.02,  0.04, ..., -0.01, -0.  ,  0.  ])

In [105]:
from sklearn.preprocessing import MinMaxScaler

sms['lda_score'] = MinMaxScaler().fit_transform(spamminess_score.reshape(-1, 1))
sms['lda_predict'] = (sms.lda_score > 0.5).astype(int)
sms['spam lda_predict lda_score'.split()].round(2).head(6)

,spam,lda_predict,lda_score
sms0,0,0,0.23
sms1,0,0,0.18
sms2!,1,1,0.72
sms3,0,0,0.18
sms4,0,0,0.29
sms5!,1,1,0.55


In [106]:
(1. - (sms.spam - sms.lda_predict).abs().sum() / len(sms)).round(3)

0.977